In [7]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from pandas import DataFrame
import os
import sys
from ipywidgets import interact
from loguru import logger
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import confusion_matrix, classification_report
from src.utils.ea.exp_funcs import classification_report_table, version_label, CompareResultsMetrics

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 00. Raw values extraction

In [8]:
def extract_raw_predictions() -> DataFrame:
    """
    Extracts the raw predictions from the dataframe.
    """
    
    df_sentiment_pred_list = []

    for n in range(3):

        df_r = pd.read_parquet(f"./../Dados/result_analyzer_{n}.parquet")
        sentiment_cols = [x for x in df_r.columns if "Sentiment_" in x]
        
        for col in sentiment_cols:
            
            dff = df_r[[col]]
            dff["n"] = n
            dff["sentiment_model"] = col
            dff.columns = ["sentiment_pred_raw","n","sentiment_model"]
            df_sentiment_pred_list.append(dff)
            
    df_sentiment_pred = pd.concat(df_sentiment_pred_list)

    values_llm = df_sentiment_pred["sentiment_pred_raw"].unique().tolist()

    negativo_values = [x for x in values_llm if "neg" in x.lower()]
    neutro_values = [x for x in values_llm if "neu" in x.lower()]
    positivo_values = [x for x in values_llm if "pos" in x.lower()]
    null = [x for x in values_llm if x!=x]
    
    return values_llm, negativo_values, neutro_values, positivo_values, null

In [9]:
values_llm, negativo_values, neutro_values, positivo_values, null = extract_raw_predictions()

values_llm, negativo_values, neutro_values, positivo_values, null

(['Negativo',
  'Neutro',
  'Positivo',
  'Neutral',
  'positivo',
  'neutro',
  'negativo',
  'neutral'],
 ['Negativo', 'negativo'],
 ['Neutro', 'Neutral', 'neutro', 'neutral'],
 ['Positivo', 'positivo'],
 [])

## 01. Function: choice dataset result

In [10]:
# results = [x for x in df_r.columns if "Sentiment__" and "pred" in x]
# results

def df_result(n:int=1)->DataFrame:
    """
    Função que carrega o dataframe de resultados do modelo de análise de sentimento

    Args:
        n (int, optional): _description_. Defaults to 1.

    Returns:
        DataFrame: _description_
    """
    
    values_llm, negativo_values, neutro_values, positivo_values, null = extract_raw_predictions()

    df_r = pd.read_parquet(f"./../Dados/result_analyzer_{n}.parquet")
    df_r["Sentiment"] = df_r["sentiment"].apply(lambda x: "Negativo" if x==0 else "Neutro" if x==1 else "Positivo")
    sentiment_cols = [x for x in df_r.columns if "Sentiment_" in x]
    for sent_col in sentiment_cols:
        df_r[f"{sent_col}_pred"] = df_r[sent_col].apply(lambda x: 0 if x in negativo_values else 1 if x in neutro_values else 2)
        
    return df_r    

    
class CompareResultsTable:
    
    
    def sample_same_model_and_version(model:str, prompt:str, execs:list[int]):

        df_ls = []
        for n in execs:

            df_r = df_result(n)
            sentiment_pred = f"Sentiment__{prompt}__{model}"
            pred = f"Sentiment__{prompt}__{model}_pred"
            just_pred = f"Justificativa__{prompt}__{model}"

            df_r_anl = df_r[["reviewId","score","sentiment", pred , sentiment_pred ,"content",just_pred]]
            keys = ["reviewId","score","sentiment","content"]
            columns = df_r_anl.drop(columns=keys).columns.tolist()
            df_r_anl.rename(columns={col: f"{col} exec {n}" for col in columns}, inplace=True)
            
            df_ls.append(df_r_anl)
            
        df_r_merge = df_ls[0].merge(df_ls[1], on=keys, how="inner")
            
        return df_r_merge

In [11]:
class_names = ["Negativo","Neutro","Positivo"]

[x + " Predito" for x in class_names]

['Negativo Predito', 'Neutro Predito', 'Positivo Predito']

## 02. Widget: choice metrics to dataset

In [37]:
modelo = "gpt-4o"
prompt = "prompt_tot"

@interact
def show_results(n=[0,1,2], modelo = ["gpt-4o-mini","gpt-4o","gpt-3.5-turbo"], prompt = ["prompt_tot","zero_shot_prompt"]):
    
    df_r = df_result(n)
    
    if n==0 and modelo=="gpt-4o-mini":
        print("")
        print(f"Modelo {modelo} não foi usado para a rodada {n}")
    else:
        pred = f"Sentiment__{prompt}__{modelo}_pred"
        just_pred = f"Justificativa__{prompt}__{modelo}"
        print(classification_report(df_r["sentiment"], df_r[pred]))
        
        
        pred = f"Sentiment__{prompt}__{modelo}_pred"
        just_pred = f"Justificativa__{prompt}__{modelo}"
        class_names = ["Negativo","Neutro","Positivo"]
        cm = confusion_matrix(df_r["sentiment"], df_r[pred])
        df_cm = pd.DataFrame(cm, index=class_names, columns= [x + " Predito" for x in class_names])
        df_cm
        
        print("\n\n")
        display(df_cm)

interactive(children=(Dropdown(description='n', options=(0, 1, 2), value=0), Dropdown(description='modelo', op…

## 03. Join result

In [38]:
metric_table_ls = []


for n in [0,1,2]:
    df_r = df_result( n)
    for modelo in ["gpt-4o-mini","gpt-4o","gpt-3.5-turbo"]:
        for prompt in  ["prompt_tot","zero_shot_prompt"]:
            if n==0 and modelo=="gpt-4o-mini":
                pass
            else:
                pred = f"Sentiment__{prompt}__{modelo}_pred"
                metrics_table = classification_report_table(df_r["sentiment"], df_r[pred], num_classes=3)
                metrics_table["model"] = modelo
                metrics_table["version"] = version_label(modelo,n) 
                metrics_table["prompt"] = prompt
                metrics_table["execution"] = n

                metric_table_ls.append(metrics_table)

In [39]:
df_results = pd.concat(metric_table_ls)
df_results

,labels,precision,recall,f1-score,support,model,version,prompt,execution
0,0,0.56,0.92,0.70,320,gpt-4o,2024-05-13,prompt_tot,0
1,1,0.43,0.16,0.24,302,gpt-4o,2024-05-13,prompt_tot,0
2,2,0.75,0.72,0.74,356,gpt-4o,2024-05-13,prompt_tot,0
0,0,0.58,0.92,0.71,320,gpt-4o,2024-05-13,zero_shot_prompt,0
1,1,0.42,0.23,0.30,302,gpt-4o,2024-05-13,zero_shot_prompt,0
2,2,0.79,0.67,0.72,356,gpt-4o,2024-05-13,zero_shot_prompt,0
0,0,0.53,0.92,0.67,320,gpt-3.5-turbo,0613,prompt_tot,0
1,1,0.44,0.21,0.28,302,gpt-3.5-turbo,0613,prompt_tot,0
2,2,0.81,0.64,0.71,356,gpt-3.5-turbo,0613,prompt_tot,0
0,0,0.53,0.94,0.67,320,gpt-3.5-turbo,0613,zero_shot_prompt,0


### 03.1 Comparison: Same model and same version

#### 03.1.1 gpt-3.5-turbo

##### A. ToT

In [137]:
comp = CompareResultsMetrics(df_results)

comparison = comp.same_model_and_version( "gpt-3.5-turbo", "0613", "prompt_tot")
comparison

,labels,recall exec 1,recall exec 0,precision exec 1,precision exec 0,f1-score exec 1,f1-score exec 0
0,0,0.92,0.92,0.53,0.53,0.67,0.67
1,1,0.20,0.21,0.42,0.44,0.27,0.28
2,2,0.64,0.64,0.80,0.81,0.71,0.71


In [173]:
model = "gpt-3.5-turbo"
prompt = "prompt_tot"
execs = [0,1]

df_re = CompareResultsTable.sample_same_model_and_version( model, prompt, execs)

pred = f"Sentiment__{prompt}__{modelo}_pred exec {execs[0]}"
pred_ = f"Sentiment__{prompt}__{modelo}_pred exec {execs[1]}"

df__ = df_re[df_re[pred] != df_re[pred_]]

logger.warning(f"Total : {df__.shape[0]}")

df_re[df_re[pred] != df_re[pred_]].head(100).reset_index(drop=True)

2025-04-08 22:51:08.430 | WARNING  | __main__:<module>:12 - Total : 25


,reviewId,score,sentiment,Sentiment__prompt_tot__gpt-3.5-turbo_pred exec 0,Sentiment__prompt_tot__gpt-3.5-turbo exec 0,content,Justificativa__prompt_tot__gpt-3.5-turbo exec 0,Sentiment__prompt_tot__gpt-3.5-turbo_pred exec 1,Sentiment__prompt_tot__gpt-3.5-turbo exec 1,Justificativa__prompt_tot__gpt-3.5-turbo exec 1
0,98d7b54c-4ad0-4999-9d83-6fe084d2298b,1,0,2,Positivo,Vou avaliar ainda... Se gostar dou 5 ☆,"O texto indica uma avaliação positiva, mencion...",1,Neutro,A avaliação mencionada não fornece informações...
1,34423d33-6fff-4bab-b05a-4beb92d436ca,2,0,1,Neutro,Legal,A análise das avaliações do aplicativo Mercado...,2,Positivo,A análise das avaliações do aplicativo Mercado...
2,02466f8e-300f-4a0d-b45f-2deccf88206e,2,0,1,Neutro,Eu não tô conseguindo acessar minha conta no M...,O comentário menciona uma dificuldade específi...,0,Negativo,O usuário está enfrentando dificuldades para a...
3,8e379bf3-79b3-42a5-a6ee-8332e227dfe2,2,0,0,Negativo,Já foi bom tá tudo caro no mercado livre e par...,O comentário menciona que tudo está caro no Me...,1,Neutro,O comentário apresenta aspectos positivos e ne...
4,f514be36-66e1-4175-be3e-835a6fcdbc86,2,0,0,Negativo,Tem tudo que eu quero com enorme variedade de ...,O usuário está insatisfeito com o valor alto d...,1,Neutro,O usuário expressa uma opinião neutra sobre o ...
5,b85530a9-aa4a-4a57-8dff-813a94e04e8d,3,1,1,Neutro,Não consigo entra no App,A frase 'Não consigo entrar no App' não expres...,0,Negativo,O usuário está expressando uma dificuldade em ...
6,c4e9bfd0-86e1-4599-8f19-2415d7804b93,3,1,0,Negativo,Gdcvvvbxxdz1ffcee Gdcvvvbxxdz1 c4rv fvz.x f vo...,A mensagem contém uma mistura de letras e núme...,1,Neutro,A mensagem fornecida não contém informações cl...
7,636cd91d-fb33-49ca-beab-fbdcab3d8c50,3,1,1,Neutro,E bom,A frase 'E bom' é ambígua e pode ser interpret...,2,Positivo,A expressão 'E bom' é uma forma simplificada d...
8,d3a62ae2-2454-44b4-9d6d-8e4174bdb195,3,1,1,Neutro,Certo Certo Certo 9999 obrigado,A frase 'Certo Certo Certo 9999 obrigado' não ...,2,Positivo,A presença da palavra 'obrigado' indica uma ex...
9,8b200fd1-e71a-4dca-9afc-2c23abcc3c54,3,1,0,Negativo,Eu daria 5 estrelas porém o meu não esta baixa...,O usuário está insatisfeito porque não consegu...,1,Neutro,"A avaliação menciona que daria 5 estrelas, o q..."


In [139]:
df_re[df_re[pred] != df_re[pred_]].query(f"`{pred}`==0 and `{pred_}`==2")

,reviewId,score,sentiment,Sentiment__prompt_tot__gpt-3.5-turbo_pred exec 0,Sentiment__prompt_tot__gpt-3.5-turbo exec 0,content,Justificativa__prompt_tot__gpt-3.5-turbo exec 0,Sentiment__prompt_tot__gpt-3.5-turbo_pred exec 1,Sentiment__prompt_tot__gpt-3.5-turbo exec 1,Justificativa__prompt_tot__gpt-3.5-turbo exec 1


In [140]:
df_re[df_re[pred] != df_re[pred_]].query(f"`{pred}`==2 and `{pred_}`==0")

,reviewId,score,sentiment,Sentiment__prompt_tot__gpt-3.5-turbo_pred exec 0,Sentiment__prompt_tot__gpt-3.5-turbo exec 0,content,Justificativa__prompt_tot__gpt-3.5-turbo exec 0,Sentiment__prompt_tot__gpt-3.5-turbo_pred exec 1,Sentiment__prompt_tot__gpt-3.5-turbo exec 1,Justificativa__prompt_tot__gpt-3.5-turbo exec 1
840,b608403f-6514-464a-b3ad-f76c50fd8711,5,2,2,Positivo,É um dos melhores mercado pode confiar,A frase indica que o mercado é confiável e um ...,0,Negativo,A frase apresentada está confusa e não transmi...


In [176]:
df_re.query(f"sentiment==2 and `{pred_}`==0")

,reviewId,score,sentiment,Sentiment__prompt_tot__gpt-3.5-turbo_pred exec 0,Sentiment__prompt_tot__gpt-3.5-turbo exec 0,content,Justificativa__prompt_tot__gpt-3.5-turbo exec 0,Sentiment__prompt_tot__gpt-3.5-turbo_pred exec 1,Sentiment__prompt_tot__gpt-3.5-turbo exec 1,Justificativa__prompt_tot__gpt-3.5-turbo exec 1
622,98c0e4f4-c3c3-4660-95ed-12b7e11d72d8,4,2,0,Negativo,Fala que o frete é grátis mas nem sempre é,O comentário indica que a empresa Mercado Livr...,0,Negativo,O comentário indica que a promessa de frete gr...
628,c4fd097b-aa02-420a-956f-5516c4a1b728,4,2,0,Negativo,"Ótima ferramenta de trabalho. Exceto, pelos ca...",O comentário menciona que a ferramenta de trab...,0,Negativo,O comentário menciona que a ferramenta de trab...
630,4e4cda9d-039e-498a-82fe-ef42f62bcc05,4,2,0,Negativo,"Um pé atrás sempre, não consigo ter confiança ...","A expressão 'Um pé atrás sempre, não consigo t...",0,Negativo,"A expressão 'Um pé atrás sempre, não consigo t..."
631,22365905-d795-4faf-a681-61cc9f57f373,4,2,0,Negativo,"Não estou conseguindo abrir o app, aparece na ...",O usuário está enfrentando um problema ao tent...,0,Negativo,O usuário está enfrentando um problema ao tent...
639,4028a7bf-3c3f-456a-8732-6d0d7e9bf1fd,4,2,0,Negativo,O app até que é razoável. Ruim mesmo é o ML nã...,O autor da avaliação expressa insatisfação com...,0,Negativo,O autor da avaliação expressa insatisfação com...
...,...,...,...,...,...,...,...,...,...,...
840,b608403f-6514-464a-b3ad-f76c50fd8711,5,2,2,Positivo,É um dos melhores mercado pode confiar,A frase indica que o mercado é confiável e um ...,0,Negativo,A frase apresentada está confusa e não transmi...
870,4f18611b-d962-4970-86ba-815627a4842a,5,2,0,Negativo,"Vou só dá Duas , isso por falar q entregariam ...",O usuário expressa insatisfação com a experiên...,0,Negativo,O usuário expressa insatisfação com a experiên...
875,75de9956-d3bc-4543-bb2a-29b3e2116499,5,2,0,Negativo,"Ótimo aplicativo, mais tem alguns dias que o a...",O usuário expressa insatisfação com a lentidão...,0,Negativo,O usuário expressa insatisfação com a lentidão...
889,caf44ddc-2097-445b-95f6-5ceef0bcc3c5,5,2,0,Negativo,"Gosto muito de comprar pelo mercado livre,tenh...",O usuário expressa insatisfação com a falta de...,0,Negativo,O usuário expressa insatisfação com a falta de...


In [177]:
df_re.query(f"sentiment==0 and `{pred_}`==2")

,reviewId,score,sentiment,Sentiment__prompt_tot__gpt-3.5-turbo_pred exec 0,Sentiment__prompt_tot__gpt-3.5-turbo exec 0,content,Justificativa__prompt_tot__gpt-3.5-turbo exec 0,Sentiment__prompt_tot__gpt-3.5-turbo_pred exec 1,Sentiment__prompt_tot__gpt-3.5-turbo exec 1,Justificativa__prompt_tot__gpt-3.5-turbo exec 1
175,efcfcb0b-48a8-4810-b7e6-262e7ce1c61b,2,0,2,Positivo,"Muito bom o aplicativo,mas se parcelasse bolet...",O comentário elogia o aplicativo do Mercado Li...,2,Positivo,"O cliente elogia o aplicativo, chamando-o de '..."
194,34423d33-6fff-4bab-b05a-4beb92d436ca,2,0,1,Neutro,Legal,A análise das avaliações do aplicativo Mercado...,2,Positivo,A análise das avaliações do aplicativo Mercado...
208,42dcae5c-e1f0-4712-9780-dd4f7ec9ed62,2,0,2,Positivo,gostei espero que eu goste das roupas,A frase 'gostei espero que eu goste das roupas...,2,Positivo,A frase 'gostei espero que eu goste das roupas...
226,da652d8c-044c-41df-b114-9fa7875def8a,2,0,2,Positivo,"Achei bom, vamos ver, se deu certo.","A expressão 'Achei bom, vamos ver, se deu cert...",2,Positivo,"A expressão 'Achei bom, vamos ver, se deu cert..."
277,fe815014-4ee6-4ea0-ada5-9c1a148285ca,2,0,2,Positivo,O melhor mercado do brasil,A expressão 'O melhor mercado do Brasil' indic...,2,Positivo,A expressão 'O melhor mercado do Brasil' indic...


##### B.Zero-shot

In [141]:
comparison = comp.same_model_and_version( "gpt-3.5-turbo", "0613", "zero_shot_prompt")
comparison

,labels,recall exec 1,recall exec 0,precision exec 1,precision exec 0,f1-score exec 1,f1-score exec 0
0,0,0.94,0.94,0.53,0.53,0.67,0.67
1,1,0.13,0.13,0.38,0.36,0.19,0.19
2,2,0.65,0.65,0.76,0.76,0.70,0.70


In [164]:
model = "gpt-3.5-turbo"
prompt = "zero_shot_prompt"
execs = [0,1]

df_re = CompareResultsTable.sample_same_model_and_version( model, prompt, execs)

pred = f"Sentiment__{prompt}__{modelo}_pred exec {execs[0]}"
pred_ = f"Sentiment__{prompt}__{modelo}_pred exec {execs[1]}"

df__ = df_re[df_re[pred] != df_re[pred_]]

logger.warning(f"Total : {df__.shape[0]}")

df_re[df_re[pred] != df_re[pred_]].head(100).reset_index(drop=True)

2025-04-08 22:29:10.365 | WARNING  | __main__:<module>:12 - Total : 12


,reviewId,score,sentiment,Sentiment__zero_shot_prompt__gpt-3.5-turbo_pred exec 0,Sentiment__zero_shot_prompt__gpt-3.5-turbo exec 0,content,Justificativa__zero_shot_prompt__gpt-3.5-turbo exec 0,Sentiment__zero_shot_prompt__gpt-3.5-turbo_pred exec 1,Sentiment__zero_shot_prompt__gpt-3.5-turbo exec 1,Justificativa__zero_shot_prompt__gpt-3.5-turbo exec 1
0,17ab8e3a-d0ac-4105-aa16-65471e225a4e,2,0,1,Neutro,Depois da ultima atualização não consigo acomp...,O usuário menciona um problema após a última a...,0,Negativo,O usuário menciona que após a última atualizaç...
1,34423d33-6fff-4bab-b05a-4beb92d436ca,2,0,1,Neutro,Legal,A expressão 'Legal' é neutra e não fornece inf...,2,Positivo,"A avaliação contém a palavra 'Legal', que gera..."
2,1594c983-3915-49f2-af21-1d4ba0f363f6,3,1,1,Neutral,Só vou dar 5 estrelas quando adicionar um chat...,O comentário menciona uma funcionalidade que o...,0,Negativo,O usuário expressa insatisfação por não ter a ...
3,3ed2771f-8cce-4162-b8f0-f570906925d6,3,1,0,Negativo,Boa porém precisam melhorar as taxas,"O texto menciona que a avaliação é boa, porém ...",1,Neutro,"O texto menciona um aspecto positivo ('Boa'), ..."
4,3bf89e8f-5d21-4f73-9d70-a6618e3d35d0,4,2,1,Neutro,ótimo gostei muito de usar só achei um pouco c...,O texto apresenta uma mistura de aspectos posi...,0,Negativo,O texto apresenta uma opinião positiva sobre a...
5,d1b29a53-8bf0-4478-9416-76b958ec1c67,4,2,1,Neutral,os pedidos poderiam chegar mais cedo,"A frase expressa uma sugestão de melhoria, ind...",0,Negativo,O comentário expressa uma insatisfação com a d...
6,6c26deb0-5bbd-4342-ac90-3e1b8ce6c81f,4,2,1,Neutro,"Ótimo e seguro, devolução do valor se solicita...",O usuário elogia a segurança e eficiência do a...,0,Negativo,O usuário elogia a segurança e eficiência do a...
7,746e61a4-d009-406d-83d4-d846784d91b0,4,2,1,Neutro,O único site de compras que eu compro sem medo...,O texto apresenta uma mistura de opiniões posi...,0,Negativo,O texto apresenta uma opinião negativa em rela...
8,d5cec7fd-90af-4dc1-ab5f-445303e36465,4,2,0,Negativo,Mercado livre. Ótima opção pra você que deseja...,O texto menciona um problema relacionado à dif...,1,Neutro,"O texto apresenta aspectos positivos, como a r..."
9,baab36ea-3147-4933-97da-66c273919937,4,2,0,Negativo,Eu nao curto baixar app de vendas pois eles te...,O texto expressa uma opinião negativa em relaç...,1,Neutro,O texto apresenta uma mistura de opiniões sobr...


In [143]:
df_re[df_re[pred] != df_re[pred_]].query(f"`{pred}`==0 and `{pred_}`==2")

,reviewId,score,sentiment,Sentiment__zero_shot_prompt__gpt-3.5-turbo_pred exec 0,Sentiment__zero_shot_prompt__gpt-3.5-turbo exec 0,content,Justificativa__zero_shot_prompt__gpt-3.5-turbo exec 0,Sentiment__zero_shot_prompt__gpt-3.5-turbo_pred exec 1,Sentiment__zero_shot_prompt__gpt-3.5-turbo exec 1,Justificativa__zero_shot_prompt__gpt-3.5-turbo exec 1


In [144]:
df_re[df_re[pred] != df_re[pred_]].query(f"`{pred}`==2 and `{pred_}`==0")

,reviewId,score,sentiment,Sentiment__zero_shot_prompt__gpt-3.5-turbo_pred exec 0,Sentiment__zero_shot_prompt__gpt-3.5-turbo exec 0,content,Justificativa__zero_shot_prompt__gpt-3.5-turbo exec 0,Sentiment__zero_shot_prompt__gpt-3.5-turbo_pred exec 1,Sentiment__zero_shot_prompt__gpt-3.5-turbo exec 1,Justificativa__zero_shot_prompt__gpt-3.5-turbo exec 1


#### 03.1.2 gpt-4o

##### A.ToT

In [145]:
comparison = comp.same_model_and_version( "gpt-4o", "2024-05-13", "prompt_tot")
comparison

,labels,recall exec 1,recall exec 0,precision exec 1,precision exec 0,f1-score exec 1,f1-score exec 0
0,0,0.94,0.92,0.55,0.56,0.70,0.70
1,1,0.15,0.16,0.46,0.43,0.23,0.24
2,2,0.72,0.72,0.76,0.75,0.74,0.74


In [165]:
model = "gpt-4o"
prompt = "prompt_tot"
execs = [0,1]

df_re = CompareResultsTable.sample_same_model_and_version( model, prompt, execs)

pred = f"Sentiment__{prompt}__{model}_pred exec {execs[0]}"
pred_ = f"Sentiment__{prompt}__{model}_pred exec {execs[1]}"

df__ = df_re[df_re[pred] != df_re[pred_]]

logger.warning(f"Total : {df__.shape[0]}")

df_re[df_re[pred] != df_re[pred_]].head(100).reset_index(drop=True)

2025-04-08 22:29:27.323 | WARNING  | __main__:<module>:12 - Total : 52


,reviewId,score,sentiment,Sentiment__prompt_tot__gpt-4o_pred exec 0,Sentiment__prompt_tot__gpt-4o exec 0,content,Justificativa__prompt_tot__gpt-4o exec 0,Sentiment__prompt_tot__gpt-4o_pred exec 1,Sentiment__prompt_tot__gpt-4o exec 1,Justificativa__prompt_tot__gpt-4o exec 1
0,fe8d3b48-c72d-41cc-a043-6bf6bd39bc09,1,0,1,Neutro,Bom dia ☀️! Paz e bem 😃! Tudo bem? 👍🏿 Espero q...,O texto contém saudações e expressões positiva...,0,Negativo,Embora a mensagem comece com saudações positiv...
1,757d85b7-0656-4e59-96f5-656295482beb,1,0,1,Neutro,"App muito bom pra compras, bons descontos e mu...",O usuário elogia o aplicativo por oferecer bon...,0,Negativo,Embora o usuário elogie o aplicativo por seus ...
2,85610e19-0a47-4e50-9951-427f371d97f1,2,0,1,Neutro,Muito bom a única coisa que deixa a desejar e ...,O comentário começa com um elogio ('Muito bom'...,0,Negativo,"Embora o comentário comece com 'Muito bom', o ..."
3,9c09d1db-3b3e-48fe-8a95-45ffea14c053,2,0,1,Neutro,O app é ótimo mas não consigo mais entrar no m...,A avaliação menciona um aspecto positivo ('O a...,0,Negativo,Embora o usuário comece elogiando o aplicativo...
4,8216cb88-0124-41bb-97b3-0e95353ad686,2,0,1,Neutro,"Tudo funciona bem, ate vc precisar entrar em c...",O comentário menciona uma experiência negativa...,0,Negativo,Embora o usuário mencione que é 'mto bom compr...
5,0fc7b2bc-4474-4034-8be1-651f6fee0f68,2,0,1,Neutro,"Bom para comprar, porém atualmente com Falhas....",A avaliação menciona aspectos positivos e nega...,0,Negativo,Embora o comentário comece com uma observação ...
6,fadabde4-48e2-41c3-99eb-ca0b89bb0891,3,1,2,positivo,E mei o rui,A frase 'E mei o rui' parece ser uma tentativa...,0,Negativo,A frase 'E mei o rui' parece ser uma tentativa...
7,55517e2e-59a5-4d55-95c3-bb0546af5ee6,3,1,1,Neutro,Boa,"A palavra 'Boa' é uma expressão positiva, mas ...",2,Positivo,A análise do texto da revisão do aplicativo Me...
8,51c617d4-76e3-4275-97ea-ed99fee73b94,3,1,2,Positivo,tirando o frete aqui pro nordeste q fica muito...,Apesar de mencionar um ponto negativo sobre o ...,0,Negativo,A avaliação menciona que o frete para o nordes...
9,0170767e-1956-46fc-8c8b-d830a88be3ec,3,1,1,Neutro,"O aplicativo até um tempo atrás era excelente,...",O usuário expressa insatisfação com problemas ...,0,Negativo,O usuário expressa insatisfação com o aplicati...


In [166]:
df_re[df_re[pred] != df_re[pred_]].query(f"`{pred}`==0 and `{pred_}`==2")

,reviewId,score,sentiment,Sentiment__prompt_tot__gpt-4o_pred exec 0,Sentiment__prompt_tot__gpt-4o exec 0,content,Justificativa__prompt_tot__gpt-4o exec 0,Sentiment__prompt_tot__gpt-4o_pred exec 1,Sentiment__prompt_tot__gpt-4o exec 1,Justificativa__prompt_tot__gpt-4o exec 1
828,d586cff0-e4e1-495a-a2f6-b4f28de4d104,5,2,0,Negativo,Bom..,A palavra 'Bom..' sugere uma hesitação ou insa...,2,Positivo,A palavra 'Bom' geralmente indica uma avaliaçã...


In [167]:
df_re[df_re[pred] != df_re[pred_]].query(f"`{pred}`==2 and `{pred_}`==0")

,reviewId,score,sentiment,Sentiment__prompt_tot__gpt-4o_pred exec 0,Sentiment__prompt_tot__gpt-4o exec 0,content,Justificativa__prompt_tot__gpt-4o exec 0,Sentiment__prompt_tot__gpt-4o_pred exec 1,Sentiment__prompt_tot__gpt-4o exec 1,Justificativa__prompt_tot__gpt-4o exec 1
328,fadabde4-48e2-41c3-99eb-ca0b89bb0891,3,1,2,positivo,E mei o rui,A frase 'E mei o rui' parece ser uma tentativa...,0,Negativo,A frase 'E mei o rui' parece ser uma tentativa...
330,51c617d4-76e3-4275-97ea-ed99fee73b94,3,1,2,Positivo,tirando o frete aqui pro nordeste q fica muito...,Apesar de mencionar um ponto negativo sobre o ...,0,Negativo,A avaliação menciona que o frete para o nordes...
615,822499d8-23fa-4ab7-b162-2f31ce4ea161,3,1,2,Positivo,Bom mas ocupa muito espaço memoria telefoneq,O comentário 'Bom mas ocupa muito espaço memor...,0,Negativo,A avaliação menciona que o aplicativo é bom ('...


##### B.Zero-shot

In [149]:
comparison = comp.same_model_and_version( "gpt-4o", "2024-05-13", "zero_shot_prompt")
comparison

,labels,recall exec 1,recall exec 0,precision exec 1,precision exec 0,f1-score exec 1,f1-score exec 0
0,0,0.95,0.92,0.55,0.58,0.70,0.71
1,1,0.20,0.23,0.44,0.42,0.27,0.30
2,2,0.66,0.67,0.79,0.79,0.72,0.72


In [168]:
model = "gpt-4o"
prompt = "zero_shot_prompt"
execs = [0,1]

df_re = CompareResultsTable.sample_same_model_and_version( model, prompt, execs)

pred = f"Sentiment__{prompt}__{model}_pred exec {execs[0]}"
pred_ = f"Sentiment__{prompt}__{model}_pred exec {execs[1]}"

df__ = df_re[df_re[pred] != df_re[pred_]]

logger.warning(f"Total : {df__.shape[0]}")

df_re[df_re[pred] != df_re[pred_]].head(100).reset_index(drop=True)

2025-04-08 22:29:35.579 | WARNING  | __main__:<module>:12 - Total : 53


,reviewId,score,sentiment,Sentiment__zero_shot_prompt__gpt-4o_pred exec 0,Sentiment__zero_shot_prompt__gpt-4o exec 0,content,Justificativa__zero_shot_prompt__gpt-4o exec 0,Sentiment__zero_shot_prompt__gpt-4o_pred exec 1,Sentiment__zero_shot_prompt__gpt-4o exec 1,Justificativa__zero_shot_prompt__gpt-4o exec 1
0,8d214b29-0554-48ab-990f-762a9fb19a65,1,0,2,positivo,Bom,A palavra 'Bom' indica uma avaliação favorável...,1,Neutro,A palavra 'Bom' é uma avaliação genérica e não...
1,fe8d3b48-c72d-41cc-a043-6bf6bd39bc09,1,0,1,Neutro,Bom dia ☀️! Paz e bem 😃! Tudo bem? 👍🏿 Espero q...,O texto contém saudações e expressões positiva...,0,Negativo,Apesar das saudações iniciais serem positivas ...
2,dfb2ffb2-34e7-4648-97a9-34c14dd1c551,1,0,1,Neutro,Compro no mercado livre há muito anos comprand...,O usuário expressa satisfação com o serviço ao...,0,Negativo,Embora o usuário tenha mencionado uma experiên...
3,85610e19-0a47-4e50-9951-427f371d97f1,2,0,1,Neutro,Muito bom a única coisa que deixa a desejar e ...,"O usuário elogia o aplicativo ('Muito bom'), m...",0,Negativo,Embora o usuário comece o comentário com 'Muit...
4,17ab8e3a-d0ac-4105-aa16-65471e225a4e,2,0,1,Neutro,Depois da ultima atualização não consigo acomp...,O usuário menciona um problema específico após...,0,Negativo,Embora o usuário mencione que o aplicativo est...
5,9c09d1db-3b3e-48fe-8a95-45ffea14c053,2,0,1,Neutro,O app é ótimo mas não consigo mais entrar no m...,O usuário elogia o aplicativo dizendo que é ót...,0,Negativo,Embora o usuário comece elogiando o aplicativo...
6,160c8d12-ce47-4e91-8840-abad0ce55343,2,0,1,Neutro,"o aplicativo é bom, porém, já veio produto tro...","O usuário menciona que o aplicativo é bom, mas...",0,Negativo,Embora o usuário comece mencionando que o apli...
7,08826a18-0313-4d24-ae78-2edf7fa85f7f,2,0,1,Neutro,gosto do app comprei meu celular por ele ótimo...,O usuário expressa satisfação com o aplicativo...,0,Negativo,Embora o usuário comece elogiando o aplicativo...
8,38dde178-1913-4a95-909c-222c77cb2ac1,2,0,1,neutro,as vezes os fornecedores são bons. mas nem sempre,O texto menciona que os fornecedores são bons ...,0,Negativo,O texto expressa uma insatisfação com a incons...
9,8216cb88-0124-41bb-97b3-0e95353ad686,2,0,1,Neutro,"Tudo funciona bem, ate vc precisar entrar em c...",O usuário menciona que tudo funciona bem e que...,0,Negativo,Embora o usuário mencione que o aplicativo fun...


In [169]:
df_re[df_re[pred] != df_re[pred_]].query(f"`{pred}`==0 and `{pred_}`==2")

,reviewId,score,sentiment,Sentiment__zero_shot_prompt__gpt-4o_pred exec 0,Sentiment__zero_shot_prompt__gpt-4o exec 0,content,Justificativa__zero_shot_prompt__gpt-4o exec 0,Sentiment__zero_shot_prompt__gpt-4o_pred exec 1,Sentiment__zero_shot_prompt__gpt-4o exec 1,Justificativa__zero_shot_prompt__gpt-4o exec 1


In [170]:
df_re[df_re[pred] != df_re[pred_]].query(f"`{pred}`==2 and `{pred_}`==0")

,reviewId,score,sentiment,Sentiment__zero_shot_prompt__gpt-4o_pred exec 0,Sentiment__zero_shot_prompt__gpt-4o exec 0,content,Justificativa__zero_shot_prompt__gpt-4o exec 0,Sentiment__zero_shot_prompt__gpt-4o_pred exec 1,Sentiment__zero_shot_prompt__gpt-4o exec 1,Justificativa__zero_shot_prompt__gpt-4o exec 1
924,de0e25ea-3c97-481c-9e5a-11f83d5deda0,5,2,2,positivo,Tem melhora muito,A frase 'Tem melhora muito' sugere que o usuár...,0,Negativo,A expressão 'Tem melhora muito' sugere que o u...


#### 03.1.3 gpt-4o-mini

##### A.ToT

In [153]:
comparison = comp.same_model_and_version( "gpt-4o-mini", "2024-07-18", "prompt_tot")
comparison

,labels,recall exec 2,recall exec 1,precision exec 2,precision exec 1,f1-score exec 2,f1-score exec 1
0,0,0.96,0.95,0.50,0.50,0.65,0.65
1,1,0.06,0.06,0.38,0.35,0.11,0.10
2,2,0.67,0.68,0.76,0.76,0.71,0.72


In [171]:
model = "gpt-4o-mini"
prompt = "prompt_tot"
execs = [1,2]

df_re = CompareResultsTable.sample_same_model_and_version( model, prompt, execs)

pred = f"Sentiment__{prompt}__{model}_pred exec {execs[0]}"
pred_ = f"Sentiment__{prompt}__{model}_pred exec {execs[1]}"

df__ = df_re[df_re[pred] != df_re[pred_]]

logger.warning(f"Total : {df__.shape[0]}")

df_re[df_re[pred] != df_re[pred_]].head(100).reset_index(drop=True)

2025-04-08 22:29:43.605 | WARNING  | __main__:<module>:12 - Total : 21


,reviewId,score,sentiment,Sentiment__prompt_tot__gpt-4o-mini_pred exec 1,Sentiment__prompt_tot__gpt-4o-mini exec 1,content,Justificativa__prompt_tot__gpt-4o-mini exec 1,Sentiment__prompt_tot__gpt-4o-mini_pred exec 2,Sentiment__prompt_tot__gpt-4o-mini exec 2,Justificativa__prompt_tot__gpt-4o-mini exec 2
0,19eef833-61fc-478b-bb1c-f4f9b1175633,1,0,1,Neutral,Ruim,A análise do review do aplicativo Mercado Livr...,0,Negativo,A análise das avaliações do aplicativo Mercado...
1,17ab8e3a-d0ac-4105-aa16-65471e225a4e,2,0,1,Neutral,Depois da ultima atualização não consigo acomp...,O usuário expressa uma insatisfação específica...,0,Negativo,Embora o usuário mencione que o aplicativo est...
2,54e3a697-d250-43f3-80ad-5a8330157b65,3,1,0,Negativo,E muito bom Só que não,A frase 'E muito bom Só que não' indica uma co...,1,Neutral,A frase 'E muito bom Só que não' apresenta uma...
3,05568dff-e4d1-46c8-bee3-84d56ae45749,3,1,2,Positivo,Muito bom a plataforma ganha no quesito entreg...,"O usuário elogia a plataforma, destacando que ...",0,Negativo,Embora o usuário reconheça que a plataforma é ...
4,7b1c20c9-a2cd-4e47-8ca5-f58b93da8d96,3,1,2,Positivo,Muito bom de extrema confiança parabéns recome...,O usuário expressa satisfação com a confiança ...,0,Negativo,Embora o usuário comece com um elogio ao Merca...
5,d46920f5-e294-4d4b-a776-817469b72545,3,1,2,Positivo,Bom na verdade eu tô esperando a câmera que eu...,O usuário expressa expectativa e otimismo em r...,1,Neutral,A revisão expressa uma expectativa em relação ...
6,c371f809-76e8-4b32-9c39-402e20a7e2d3,3,1,0,Negativo,Imi,A análise da revisão do aplicativo Mercado Liv...,2,Positivo,A análise das avaliações do aplicativo Mercado...
7,d7164015-e56f-4553-8223-75f738e0bff1,3,1,2,Positivo,Boa,A análise das avaliações do aplicativo Mercado...,1,Neutral,A análise da revisão do aplicativo Mercado Liv...
8,3c622392-2de7-4cb5-925f-7f0fdb7816aa,3,1,1,Neutral,pronto essa é minha primeira compra se chegar ...,A revisão expressa uma expectativa em relação ...,2,positivo,O usuário expressa uma expectativa positiva em...
9,77f12649-6754-4157-9c4c-90a6d0e7a4e2,3,1,2,Positivo,Primeira experiência por enquanto boa tirando ...,"A revisão menciona que a experiência foi boa, ...",0,Negativo,Embora a pessoa mencione que a experiência foi...


In [155]:
df_re[df_re[pred] != df_re[pred_]].query(f"`{pred}`==0 and `{pred_}`==2")

,reviewId,score,sentiment,Sentiment__prompt_tot__gpt-4o-mini_pred exec 1,Sentiment__prompt_tot__gpt-4o-mini exec 1,content,Justificativa__prompt_tot__gpt-4o-mini exec 1,Sentiment__prompt_tot__gpt-4o-mini_pred exec 2,Sentiment__prompt_tot__gpt-4o-mini exec 2,Justificativa__prompt_tot__gpt-4o-mini exec 2
472,c371f809-76e8-4b32-9c39-402e20a7e2d3,3,1,0,Negativo,Imi,A análise da revisão do aplicativo Mercado Liv...,2,Positivo,A análise das avaliações do aplicativo Mercado...
795,873b0c09-d1db-4007-bebd-d2f98e325e64,4,2,0,Negativo,Sempre compro pelo ML e nunca tive problemas. ...,Embora o usuário mencione que nunca teve probl...,2,Positivo,O usuário expressa satisfação ao afirmar que n...


In [156]:
df_re[df_re[pred] != df_re[pred_]].query(f"`{pred}`==2 and `{pred_}`==0")

,reviewId,score,sentiment,Sentiment__prompt_tot__gpt-4o-mini_pred exec 1,Sentiment__prompt_tot__gpt-4o-mini exec 1,content,Justificativa__prompt_tot__gpt-4o-mini exec 1,Sentiment__prompt_tot__gpt-4o-mini_pred exec 2,Sentiment__prompt_tot__gpt-4o-mini exec 2,Justificativa__prompt_tot__gpt-4o-mini exec 2
380,05568dff-e4d1-46c8-bee3-84d56ae45749,3,1,2,Positivo,Muito bom a plataforma ganha no quesito entreg...,"O usuário elogia a plataforma, destacando que ...",0,Negativo,Embora o usuário reconheça que a plataforma é ...
442,7b1c20c9-a2cd-4e47-8ca5-f58b93da8d96,3,1,2,Positivo,Muito bom de extrema confiança parabéns recome...,O usuário expressa satisfação com a confiança ...,0,Negativo,Embora o usuário comece com um elogio ao Merca...
493,77f12649-6754-4157-9c4c-90a6d0e7a4e2,3,1,2,Positivo,Primeira experiência por enquanto boa tirando ...,"A revisão menciona que a experiência foi boa, ...",0,Negativo,Embora a pessoa mencione que a experiência foi...


##### B.Zero-shot

In [157]:
comparison = comp.same_model_and_version("gpt-4o-mini", "2024-07-18", "zero_shot_prompt")
comparison

,labels,recall exec 2,recall exec 1,precision exec 2,precision exec 1,f1-score exec 2,f1-score exec 1
0,0,0.96,0.96,0.52,0.52,0.68,0.68
1,1,0.08,0.07,0.43,0.42,0.13,0.12
2,2,0.72,0.73,0.76,0.76,0.74,0.74


In [172]:
model = "gpt-4o-mini"
prompt = "zero_shot_prompt"
execs = [1,2]

df_re = CompareResultsTable.sample_same_model_and_version( model, prompt, execs)

pred = f"Sentiment__{prompt}__{model}_pred exec {execs[0]}"
pred_ = f"Sentiment__{prompt}__{model}_pred exec {execs[1]}"

df__ = df_re[df_re[pred] != df_re[pred_]]

logger.warning(f"Total : {df__.shape[0]}")

df_re[df_re[pred] != df_re[pred_]].head(100).reset_index(drop=True)

2025-04-08 22:29:53.631 | WARNING  | __main__:<module>:12 - Total : 11


,reviewId,score,sentiment,Sentiment__zero_shot_prompt__gpt-4o-mini_pred exec 1,Sentiment__zero_shot_prompt__gpt-4o-mini exec 1,content,Justificativa__zero_shot_prompt__gpt-4o-mini exec 1,Sentiment__zero_shot_prompt__gpt-4o-mini_pred exec 2,Sentiment__zero_shot_prompt__gpt-4o-mini exec 2,Justificativa__zero_shot_prompt__gpt-4o-mini exec 2
0,ff6c004c-e9cb-4af2-8baa-c406bb8b8a93,3,1,1,Neutro,Primeira vez que comprei,A frase 'Primeira vez que comprei' não express...,2,positivo,A frase 'Primeira vez que comprei' sugere uma ...
1,9fd7346b-204a-4be7-80dd-a07a55e26001,3,1,2,positivo,Na minha opinião demorou um bom tempo para atu...,"Apesar de mencionar que a atualização demorou,...",1,Neutral,O usuário expressa uma crítica sobre a demora ...
2,ed9d4328-b3a7-4412-b5d0-79d6a0c3cea9,3,1,0,Negativo,b bj,O texto 'b bj' não contém informações claras o...,1,Neutro,O texto 'b bj' não contém informações suficien...
3,7b1c20c9-a2cd-4e47-8ca5-f58b93da8d96,3,1,2,positivo,Muito bom de extrema confiança parabéns recome...,O usuário expressa satisfação com a confiança ...,0,Negativo,Embora o usuário comece com um elogio ao Merca...
4,4d1393c7-7e91-425d-ad3f-ea7cc08b38a2,3,1,1,Neutral,Estou esperando ó meu produto,A frase expressa uma expectativa em relação a ...,0,Negativo,A frase expressa uma expectativa de forma ansi...
5,34be4957-db2c-4550-b978-5b0f82523ee0,3,1,0,Negativo,"No começo estava muito bom, a opção Full era ó...",O usuário expressa insatisfação com mudanças n...,2,positivo,Apesar de mencionar problemas com a opção Full...
6,4896c152-6b80-4058-86c1-59f16e4de71e,3,1,0,negativo,"o app é bom, mas falta muita ênfase na parte d...","Embora o usuário reconheça que o app é bom, a ...",1,neutro,A revisão expressa uma opinião mista. Embora o...
7,71887fab-5e16-468a-9f77-4fdbd9ca16fc,4,2,2,positivo,"Gosto muito de comprar pelo ML, só não vou dar...",O usuário expressa satisfação com a experiênci...,0,Negativo,O usuário expressa insatisfação com os valores...
8,0f50bdcd-05d4-406f-875d-e526e8464694,4,2,2,positivo,Para o app ser perfeito precisa só ter a opção...,O usuário expressa amor pelo Mercado Livre e f...,0,Negativo,Embora o usuário expresse amor pelo Mercado Li...
9,15cbda7b-4a5c-4760-9dd4-d3e3911fa24f,4,2,2,positivo,"O APP é bom, poderiam melhorar somente o rastr...","O usuário considera o aplicativo bom, mas suge...",0,Negativo,"Embora o usuário considere o aplicativo 'bom',..."


In [159]:
df_re[df_re[pred] != df_re[pred_]].query(f"`{pred}`==0 and `{pred_}`==2")

,reviewId,score,sentiment,Sentiment__zero_shot_prompt__gpt-4o-mini_pred exec 1,Sentiment__zero_shot_prompt__gpt-4o-mini exec 1,content,Justificativa__zero_shot_prompt__gpt-4o-mini exec 1,Sentiment__zero_shot_prompt__gpt-4o-mini_pred exec 2,Sentiment__zero_shot_prompt__gpt-4o-mini exec 2,Justificativa__zero_shot_prompt__gpt-4o-mini exec 2
599,34be4957-db2c-4550-b978-5b0f82523ee0,3,1,0,Negativo,"No começo estava muito bom, a opção Full era ó...",O usuário expressa insatisfação com mudanças n...,2,positivo,Apesar de mencionar problemas com a opção Full...


In [160]:
df_re[df_re[pred] != df_re[pred_]].query(f"`{pred}`==2 and `{pred_}`==0")

,reviewId,score,sentiment,Sentiment__zero_shot_prompt__gpt-4o-mini_pred exec 1,Sentiment__zero_shot_prompt__gpt-4o-mini exec 1,content,Justificativa__zero_shot_prompt__gpt-4o-mini exec 1,Sentiment__zero_shot_prompt__gpt-4o-mini_pred exec 2,Sentiment__zero_shot_prompt__gpt-4o-mini exec 2,Justificativa__zero_shot_prompt__gpt-4o-mini exec 2
442,7b1c20c9-a2cd-4e47-8ca5-f58b93da8d96,3,1,2,positivo,Muito bom de extrema confiança parabéns recome...,O usuário expressa satisfação com a confiança ...,0,Negativo,Embora o usuário comece com um elogio ao Merca...
681,71887fab-5e16-468a-9f77-4fdbd9ca16fc,4,2,2,positivo,"Gosto muito de comprar pelo ML, só não vou dar...",O usuário expressa satisfação com a experiênci...,0,Negativo,O usuário expressa insatisfação com os valores...
692,0f50bdcd-05d4-406f-875d-e526e8464694,4,2,2,positivo,Para o app ser perfeito precisa só ter a opção...,O usuário expressa amor pelo Mercado Livre e f...,0,Negativo,Embora o usuário expresse amor pelo Mercado Li...
793,15cbda7b-4a5c-4760-9dd4-d3e3911fa24f,4,2,2,positivo,"O APP é bom, poderiam melhorar somente o rastr...","O usuário considera o aplicativo bom, mas suge...",0,Negativo,"Embora o usuário considere o aplicativo 'bom',..."
825,f05cd0e6-dad2-45f0-8866-6f72cd42e3a1,5,2,2,positivo,Bom compro sempre até hoje só uma compra minha...,O usuário menciona que sempre compra no Mercad...,0,Negativo,O usuário menciona que sempre compra no aplica...


### 03.2 Comparison: Same model and different version

- Escolhendo as duas últimas para realizar a comparação

#### 03.2.1 gpt-3.5-turbo

In [52]:
model = "gpt-3.5-turbo"
comp.same_model_and_diff_version(model, "prompt_tot")

,labels,recall exec 2 (0125),recall exec 1 (0613),precision exec 2 (0125),precision exec 1 (0613),f1-score exec 2 (0125),f1-score exec 1 (0613)
0,0,0.93,0.92,0.53,0.53,0.67,0.67
1,1,0.20,0.20,0.43,0.42,0.27,0.27
2,2,0.64,0.64,0.80,0.80,0.71,0.71


In [53]:
model = "gpt-3.5-turbo"
comp.same_model_and_diff_version(model, "zero_shot_prompt")

,labels,recall exec 2 (0125),recall exec 1 (0613),precision exec 2 (0125),precision exec 1 (0613),f1-score exec 2 (0125),f1-score exec 1 (0613)
0,0,0.94,0.94,0.53,0.53,0.67,0.67
1,1,0.13,0.13,0.38,0.38,0.20,0.19
2,2,0.65,0.65,0.76,0.76,0.70,0.70


#### 03.2.2 gpt-4o

In [54]:
model = "gpt-4o"
comp.same_model_and_diff_version(model, "prompt_tot")

,labels,recall exec 2 (2024-08-06),recall exec 1 (2024-05-13),precision exec 2 (2024-08-06),precision exec 1 (2024-05-13),f1-score exec 2 (2024-08-06),f1-score exec 1 (2024-05-13)
0,0,0.94,0.94,0.55,0.55,0.69,0.70
1,1,0.14,0.15,0.44,0.46,0.21,0.23
2,2,0.73,0.72,0.77,0.76,0.75,0.74


In [55]:
model = "gpt-4o"
comp.same_model_and_diff_version(model, "zero_shot_prompt")

,labels,recall exec 2 (2024-08-06),recall exec 1 (2024-05-13),precision exec 2 (2024-08-06),precision exec 1 (2024-05-13),f1-score exec 2 (2024-08-06),f1-score exec 1 (2024-05-13)
0,0,0.95,0.95,0.55,0.55,0.70,0.70
1,1,0.20,0.20,0.45,0.44,0.27,0.27
2,2,0.67,0.66,0.79,0.79,0.72,0.72
